In [81]:
import pandas as pd
import re 

cost = pd.read_excel("/Users/mohammad/XAV/Departemant Financial/Profit ( Discount)/هزینه تولید سوله.xlsx", sheet_name="هزینه تولید")
fi_forosh = pd.read_excel("/Users/mohammad/XAV/Departemant Financial/Profit ( Discount)/هزینه تولید سوله.xlsx", sheet_name="فی فروش")
aban = pd.read_excel("/Users/mohammad/XAV/Departemant Financial/Profit ( Discount)/هزینه تولید سوله.xlsx", sheet_name="ابان")
mehr = pd.read_excel("/Users/mohammad/XAV/Departemant Financial/Profit ( Discount)/هزینه تولید سوله.xlsx", sheet_name="مهر")
shahrivar = pd.read_excel("/Users/mohammad/XAV/Departemant Financial/Profit ( Discount)/هزینه تولید سوله.xlsx", sheet_name="شهریور")
mordad = pd.read_excel("/Users/mohammad/XAV/Departemant Financial/Profit ( Discount)/هزینه تولید سوله.xlsx", sheet_name="مرداد")
tir = pd.read_excel("/Users/mohammad/XAV/Departemant Financial/Profit ( Discount)/هزینه تولید سوله.xlsx", sheet_name="تیر")
sale_1402 = pd.read_excel("/Users/mohammad/XAV/Departemant Sale/Project /فروش نيمه اول 1402.xlsx", sheet_name="فروش")


## Hypothetically

In [82]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import random
from bokeh.models import FuncTickFormatter
from bokeh.models import Label

def num_seperator(number):  
    number_str = str(number)
    result = []
    while number_str:
        result.append(number_str[-3:])
        number_str = number_str[:-3]

    result = result[::-1]  # Reverse the list to get the groups in the correct order
    result_str = ','.join(result)  # Join the groups with a comma
    return result_str


# changing the name of goods
def extract_persian_name(input_string):
    persian_pattern = r'[\u0600-\u06FF\s]+'
    persian_name = re.findall(persian_pattern, input_string)
    persian_name = ' '.join(persian_name)
    persian_name = persian_name.replace('سورت شده', '')
    persian_name =  re.sub(' +', ' ', persian_name)

    return persian_name

fi_forosh["عنوان كالا/خدمت"] = fi_forosh['عنوان كالا/خدمت'].apply(extract_persian_name)
aban["عنوان كالا"] = aban["عنوان كالا"].apply(extract_persian_name)
mehr["عنوان كالا"] = mehr["عنوان كالا"].apply(extract_persian_name)
shahrivar["عنوان كالا"] = shahrivar["عنوان كالا"].apply(extract_persian_name)
mordad["عنوان كالا"] = mordad["عنوان كالا"].apply(extract_persian_name)
tir["عنوان كالا"] = tir["عنوان كالا"].apply(extract_persian_name)

# turning fi forosh to a dict 
fi_forosh_dict = {}
for i in range(len(fi_forosh)):
    name = fi_forosh.loc[i,"عنوان كالا/خدمت"] 
    fi_forosh_dict[name] = fi_forosh.loc[i, "فی 1KG"]


def profit(df,mah):
    # creating a dict for each good
    value_dict = {}
    for line in df["عنوان كالا"].to_list():
        hajm = cost[cost["ماه"] == mah]["تولید کل"].to_list()[0]
        fi = fi_forosh_dict[line]
        baha = df[df["عنوان كالا"] == line]["ورودي-في متوسط"].to_list()[0]
        production_cost = cost[cost["ماه"] == mah]["هزینه تولید"].to_list()[0]
        non_production_cost = cost[cost["ماه"] == mah]["هزینه متفرقه"].to_list()[0]
        packaging_cost = cost[cost["ماه"] == mah]["بسته بندی"].to_list()[0]

        profit  = int((hajm * fi) - ((baha*hajm)+production_cost+non_production_cost+production_cost))
        profit_margin = ((fi - ((baha)+(production_cost/hajm)+(packaging_cost/hajm)+(non_production_cost/hajm))) / (fi)) * 100

        value_dict[line] = {"profit" : profit, "profit_margin": round(profit_margin,2), "hajm" :hajm, 
                            "fi": fi, "baha": baha, "production_cost":production_cost, "non_production_cost":non_production_cost,
                              "packaging_cost":packaging_cost}

    # Extracting the keys and the values for 'profit' or 'profit_margin'
    series = list(value_dict.keys())
    # profits = [entry['profit'] for entry in aban_dict.values()]  # or profits = [data[company]['profit'] for company in companies]
    profit_margins = [entry['profit_margin'] for entry in value_dict.values()]

    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(series))]

    # changing the nam of month in the plot 
    if mah == 1:
        mah = "Farvardin"
    elif mah == 2 :
        mah = "Ordibehesht"
    elif mah == 3:
        mah = "Khordad"
    elif mah == 4:
        mah = "Tir"
    elif mah == 5:
        mah = "Mordad"
    elif mah == 6:
        mah = "shahrivar"

    elif mah == 7:
        mah = "Mehr"
    elif mah == 8:
        mah = "Aban"

    p = figure(x_range=series, height=600, width= 1000, title=f"Profit Margins of Each Line in {mah}",
            toolbar_location=None, tools="")

    bars = p.vbar(x=series, top=profit_margins, width=0.9, color=colors)
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.xaxis.major_label_orientation = 1.2
    # p.yaxis.formatter = FuncTickFormatter(code="""
    #     return (tick.toString().replace(/(\\d)(?=(\\d{3})+(?!\\d))/g, "$1,"))
    # """)
    for i, value in enumerate(profit_margins):
        label = Label(x=i, y=value, text=str(value), text_baseline='middle', text_align='center',text_font_size="7pt",
                        x_offset=20, y_offset= 12)
        p.add_layout(label)

    output_notebook()
    show(p)



    # plotting Profit Rial
    series = list(value_dict.keys())
    profit_margins = [entry['profit'] for entry in value_dict.values()]

    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(series))]


    p = figure(x_range=series, height=700, width= 1000, title=f"Profit of Each Line in {mah}",
            toolbar_location=None, tools="")

    bars = p.vbar(x=series, top=profit_margins, width=0.9, color=colors)
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.xaxis.major_label_orientation = 1.2
    p.yaxis.formatter = FuncTickFormatter(code="""
        return (tick.toString().replace(/(\\d)(?=(\\d{3})+(?!\\d))/g, "$1,"))
    """)
    for i, value in enumerate(profit_margins):
        label = Label(x=i, y=value, text=num_seperator(value), text_baseline='middle', text_align='center',text_font_size="5pt",
                        x_offset=30, y_offset= 15,  angle=0.285)
        p.add_layout(label)

    output_notebook()
    show(p)

    df = pd.DataFrame(value_dict).T  # Transpose the DataFrame to have the keys as index
    df.to_excel(f'profit_each_line_{mah}.xlsx', index=True)  # Set index=False if you don't want to include the index in the Excel file



profit(aban,8) 
profit(mehr,7)
profit(shahrivar,6)
profit(mordad,5)
profit(tir,4)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## Real

In [84]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import random
from bokeh.models import FuncTickFormatter
from bokeh.models import Label

def num_seperator(number):  
    number_str = str(number)
    result = []
    while number_str:
        result.append(number_str[-3:])
        number_str = number_str[:-3]

    result = result[::-1]  # Reverse the list to get the groups in the correct order
    result_str = ','.join(result)  # Join the groups with a comma
    return result_str


# changing the name of goods
def extract_persian_name(input_string):
    persian_pattern = r'[\u0600-\u06FF\s]+'
    persian_name = re.findall(persian_pattern, input_string)
    persian_name = ' '.join(persian_name)
    persian_name = persian_name.replace('سورت شده', '')
    persian_name =  re.sub(' +', ' ', persian_name)

    return persian_name

fi_forosh["عنوان كالا/خدمت"] = fi_forosh['عنوان كالا/خدمت'].apply(extract_persian_name)
aban["عنوان كالا"] = aban["عنوان كالا"].apply(extract_persian_name)
mehr["عنوان كالا"] = mehr["عنوان كالا"].apply(extract_persian_name)
shahrivar["عنوان كالا"] = shahrivar["عنوان كالا"].apply(extract_persian_name)
mordad["عنوان كالا"] = mordad["عنوان كالا"].apply(extract_persian_name)
tir["عنوان كالا"] = tir["عنوان كالا"].apply(extract_persian_name)
sale_1402["كالا/خدمت"] = sale_1402["كالا/خدمت"].apply(extract_persian_name)
sale_1402 = sale_1402[sale_1402["كد گروه فروش"].isin([2,3])]

# turning fi forosh to a dict 
fi_forosh_dict = {}
for i in range(len(fi_forosh)):
    name = fi_forosh.loc[i,"عنوان كالا/خدمت"] 
    fi_forosh_dict[name] = fi_forosh.loc[i, "فی 1KG"]


# creating a dict of portion of sale for each line
portion_dict = {}
sale_aban = sale_1402[sale_1402["ماه"]==8]
total_kg = sum(sale_aban["مقدار-فرعي"].to_list())/1000

for i in set(sale_aban["كالا/خدمت"].to_list()):
    kg = sum(sale_aban[sale_aban["كالا/خدمت"]==i]["مقدار-فرعي"].to_list())/1000
    ratio = (kg/total_kg) 
    portion_dict[i] = ratio



def profit_real(df,mah):
    value_dict = {}
    for i in df["عنوان كالا"].to_list():
        try:
            hajm = cost[cost["ماه"] == mah]["تولید کل"].to_list()[0]
            each_line_ratio = portion_dict[i]
            each_line_kg = hajm * each_line_ratio
            fi = fi_forosh_dict[i]
            baha = df[df["عنوان كالا"] == i]["ورودي-في متوسط"].to_list()[0]
            production_cost = cost[cost["ماه"] == mah]["هزینه تولید"].to_list()[0]
            non_production_cost = cost[cost["ماه"] == mah]["هزینه متفرقه"].to_list()[0]
            packaging_cost = cost[cost["ماه"] == mah]["بسته بندی"].to_list()[0]

            profit = int(( (each_line_kg) * fi ) - ( (baha * each_line_kg) + (production_cost * each_line_ratio) + (non_production_cost* each_line_ratio) + (packaging_cost*each_line_ratio) ))
            profit_margin = ((fi - ((baha)+((production_cost * each_line_ratio)/each_line_kg)+((packaging_cost * each_line_ratio)/each_line_kg )+((non_production_cost * each_line_ratio)/each_line_kg))) / (fi)) * 100

            value_dict[i] = {"profit" : profit, "profit_margin": round(profit_margin,2), "Total_Output" :hajm, 
                                "fi": fi, "baha": baha, "Ratio": each_line_ratio, "Line_kg": each_line_kg ,"production_cost":production_cost*each_line_ratio,
                                "non_production_cost":non_production_cost*each_line_ratio,
                                    "packaging_cost":packaging_cost*each_line_ratio}
        except:
            continue
            
    # Extracting the keys and the values for 'profit' or 'profit_margin'
    series = list(value_dict.keys())
    # profits = [entry['profit'] for entry in aban_dict.values()]  # or profits = [data[company]['profit'] for company in companies]
    profit_margins = [entry['profit_margin'] for entry in value_dict.values()]

    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(series))]

    # changing the nam of month in the plot 
    if mah == 1:
        mah = "Farvardin"
    elif mah == 2 :
        mah = "Ordibehesht"
    elif mah == 3:
        mah = "Khordad"
    elif mah == 4:
        mah = "Tir"
    elif mah == 5:
        mah = "Mordad"
    elif mah == 6:
        mah = "shahrivar"

    elif mah == 7:
        mah = "Mehr"
    elif mah == 8:
        mah = "Aban"
    p = figure(x_range=series, height=600, width= 1000, title=f"Profit Margins of Each Line in {mah}",
            toolbar_location=None, tools="")

    bars = p.vbar(x=series, top=profit_margins, width=0.9, color=colors)
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.xaxis.major_label_orientation = 1.2
    # p.yaxis.formatter = FuncTickFormatter(code="""
    #     return (tick.toString().replace(/(\\d)(?=(\\d{3})+(?!\\d))/g, "$1,"))
    # """)
    for i, value in enumerate(profit_margins):
        label = Label(x=i, y=value, text=str(value), text_baseline='middle', text_align='center',text_font_size="7pt",
                        x_offset=20, y_offset= 12)
        p.add_layout(label)

    output_notebook()
    show(p)



    # plotting Profit Rial
    series = list(value_dict.keys())
    profit_margins = [entry['profit'] for entry in value_dict.values()]

    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(series))]


    p = figure(x_range=series, height=700, width= 1000, title=f"Real Profit of Each Line in {mah}",
            toolbar_location=None, tools="")

    bars = p.vbar(x=series, top=profit_margins, width=0.9, color=colors)
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.xaxis.major_label_orientation = 1.2
    p.yaxis.formatter = FuncTickFormatter(code="""
        return (tick.toString().replace(/(\\d)(?=(\\d{3})+(?!\\d))/g, "$1,"))
    """)
    for i, value in enumerate(profit_margins):
        label = Label(x=i, y=value, text=num_seperator(value), text_baseline='middle', text_align='center',text_font_size="7pt",
                        x_offset=30, y_offset= 15)
        p.add_layout(label)

    output_notebook()
    show(p)

    df = pd.DataFrame(value_dict).T  # Transpose the DataFrame to have the keys as index
    df.to_excel(f'profit_each_line_real_{mah}.xlsx', index=True)  # Set index=False if you don't want to include the index in the Excel file



profit_real(aban,8) 
profit_real(mehr,7)
profit_real(shahrivar,6)
profit_real(mordad,5)
profit_real(tir,4)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...